In [64]:
import wikipedia
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import re
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from stop_words import get_stop_words
from nltk.corpus import stopwords


org_names = ['yandex', 'apple', 'samsung', 'mercedes',
             'Oracle', 'Walt Disney', 'General Electric']

y_dict = {}

for (i, org) in enumerate(org_names):
    y_dict[org] = i

print(y_dict)

pages_list = []

stop_words = list(get_stop_words('ru'))
nltk_words = list(stopwords.words('russian'))
stop_words.extend(nltk_words)

# initializing features and samples lists
X = []
y = []

def collect_data(org_names):
    # setting language of wiki pages to Russian
    wikipedia.set_lang('Ru')


    # getting pages describing our organizations
    for org in org_names:
        org_pages = wikipedia.search(org, results=17)
        for page in org_pages:
            pages_list.append((page, org))

    # collection features and samples
    for org_page, org_name in pages_list:
        # print('Org_page: ', org_page)
        # print('Summary: ', (wikipedia.page(org_name)).summary)
        # print('=======')
        X.append((wikipedia.page(org_page)).summary)
        y.append(y_dict[str(org_name)])

    return X, y

def text_processing(X):
    new_X = []

    for i in X:
        doc = re.sub(r'\d', ' ', str(i))

        doc = re.sub(r'\s+[a-zA-Z]\s+', ' ', doc)

        doc = re.sub(r'\^[a-zA-Z]\s+', ' ', doc)

        doc = re.sub(r'\s+', ' ', doc, flags=re.I)

        doc = re.sub('\n', ' ', doc)

        doc = doc.lower()

        new_X.append(doc)

    return new_X

data = collect_data(org_names)
preprocessed_data = text_processing(data[0])

X_train, X_test, y_train, y_test = train_test_split(preprocessed_data, data[1])
vectorizer = CountVectorizer()
X_train_transformed = vectorizer.fit_transform(X_train)
X_test_transformed = vectorizer.transform(X_test)

pipe = Pipeline([('vect', CountVectorizer(min_df = 3, stop_words=stop_words)),
                ('tfidf', TfidfTransformer()),
                ('clf', SVC(random_state=1))])

parameters = {'vect__ngram_range': [(1, 1), (1, 2)],
             'tfidf__use_idf': (True, False),
             'clf__C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}

grid_s = GridSearchCV(pipe, parameters, n_jobs=-1, verbose=1)

grid_s_fit = grid_s.fit(X_train, y_train)

y_dict_trans = {}

for (org, i) in enumerate(org_names):
    y_dict_trans[org] = i

predicted = grid_s_fit.predict(X_test)

accuracy_score(y_test, predicted)

{'yandex': 0, 'apple': 1, 'samsung': 2, 'mercedes': 3, 'Oracle': 4, 'Walt Disney': 5, 'General Electric': 6}


Z:\anaconda\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 3 folds for each of 28 candidates, totalling 84 fits


[Parallel(n_jobs=-1)]: Done  84 out of  84 | elapsed:    1.5s finished
Z:\anaconda\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
Z:\anaconda\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.9666666666666667

In [88]:
y_dict_trans[grid_s_fit.predict(['корея'])[0]]


'Oracle'

In [66]:
data[0]

['«Я́ндекс» — российская транснациональная компания, зарегистрированная в Нидерландах и владеющая одноимённой системой поиска в Сети, интернет-порталами и службами в нескольких странах. Наиболее заметное положение занимает на рынках России, Турции, Белоруссии и Казахстана.\nПоисковая система «Яндекс» является четвёртой среди поисковых систем мира по количеству обрабатываемых поисковых запросов (свыше 6,3 млрд в месяц на начало 2014 года). По состоянию на сентябрь 2018 года, согласно рейтингу Alexa.com, сайт yandex.ru по популярности занимает 21-е место в мире и 1-е — в России.\nПоисковая система Yandex.ru была официально анонсирована 23 сентября 1997 года и первое время развивалась в рамках компании CompTek International. Как отдельная компания «Яндекс» образовалась в 2000 году. В мае 2011 года Яндекс провёл первичное размещение акций, заработав на этом больше, чем какая-либо из Интернет-компаний со времён IPO-поисковика Google в 2004 году.\nПриоритетное направление компании — разработ

In [89]:
import pickle

In [90]:
pickle.dump(grid_s_fit, open('model.sav', 'wb'))

In [91]:
load_model = pickle.load(open('model.sav', 'rb'))

In [92]:
load_model

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=3,
                                                        ngram_range=(1, 1),
                                       